This notebook 
- Is based on the notebook from  Databricks documentation https://docs.databricks.com/aws/en/generative-ai/create-query-vector-search?language=Python%C2%A0SDK

- Direct link to original notebook https://docs.databricks.com/aws/en/notebooks/source/generative-ai/vector-search-python-sdk-example.html

Some of the key updates to the original notebook
- Using python sdk to update the table, run a sync and verify that the index was updated

# Vector Search Python SDK example usage

This notebook shows how to use the Vector Search Python SDK, which provides a `VectorSearchClient` as a primary API for working with Vector Search.

Alternatively, you can call the REST API directly.

## Requirements

This notebook assumes that a Model Serving endpoint named `databricks-gte-large-en` exists. To create that endpoint, see the notebook "Call a GTE embeddings model using Mosaic AI Model Serving" ([AWS](https://docs.databricks.com/en/generative-ai/create-query-vector-search.html#embedding-model-notebooks) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/create-query-vector-search#embedding-model-notebooks) | [GCP](https://docs.databricks.com/gcp/en/generative-ai/create-query-vector-search.html#embedding-model-notebooks)).

In [0]:
%pip install --upgrade --force-reinstall databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
from databricks.vector_search.client import VectorSearchClient

vsc = VectorSearchClient()

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


In [0]:
help(VectorSearchClient)

Help on class VectorSearchClient in module databricks.vector_search.client:

class VectorSearchClient(builtins.object)
 |  VectorSearchClient(workspace_url=None, personal_access_token=None, service_principal_client_id=None, service_principal_client_secret=None, azure_tenant_id=None, azure_login_id='2ff814a6-3304-4ab8-85cb-cd0e6f879c1d', disable_notice=False, credential_strategy=None)
 |
 |  A client for interacting with the Vector Search service.
 |
 |  This client provides methods for managing endpoints and indexes in the Vector Search service.
 |
 |  Methods defined here:
 |
 |  __init__(self, workspace_url=None, personal_access_token=None, service_principal_client_id=None, service_principal_client_secret=None, azure_tenant_id=None, azure_login_id='2ff814a6-3304-4ab8-85cb-cd0e6f879c1d', disable_notice=False, credential_strategy=None)
 |      Initialize the VectorSearchClient.
 |
 |      :param str workspace_url: The URL of the workspace.
 |      :param str personal_access_token: The 

## Load toy dataset into source Delta table

The following creates the source Delta table.

In [0]:

# Specify the catalog and schema to use. You must have USE_CATALOG privilege on the catalog and USE_SCHEMA and CREATE_TABLE privileges on the schema.
# Change the catalog and schema here if necessary.

catalog_name = "main"
schema_name = "default"


In [0]:
source_table_name = "en_wiki"
source_table_fullname = f"{catalog_name}.{schema_name}.{source_table_name}"

In [0]:
# Uncomment if you want to start from scratch.

# spark.sql(f"DROP TABLE {source_table_fullname}")

In [0]:
source_df = spark.read.parquet("/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet").limit(10)
display(source_df)

title id revisionId revisionTimestamp revisionUsername revisionUsernameId text Pretoria 25002 683603300 2015-10-01T09:37:51Z CommonsDelinker 2304267 {{Other uses|Pretoria (disambiguation)}} {{Use South African English|date=May 2012}} {{Use dmy dates|date=May 2011}} {{Infobox South African town 2011 | name = Pretoria | nickname = Jacaranda City, PTA | image_skyline = File:Montage_of_Pretoria.jpg | imagesize = 350px | image_caption = Clockwise from top left: Pretoria CBD skyline, Front view of the [[Union Buildings]], [[Voortrekker Monument]], Administration Building of the [[University of Pretoria]], [[Church Square, Pretoria|Church Square]], [[Loftus Versfeld Stadium]] and the [[Palace of Justice (South Africa)|Palace of Justice]]. | motto = Præstantia Prævaleat Prætoria (May Pretoria Be Pre-eminent In Excellence) | image_flag = Flag of Pretoria.svg | flag_link = Flag of Pretoria | image_seal = Pretoria_coa.jpg | latd = 25 |latm = 44 |lats = 46 | longd = 28 |longm = 11 |longs = 17 | province = Gauteng | district = | municipality = City of Tshwane | established_date = 1855 | government_type = <!-- Ward 00 --> | leader_party = | leader_title = Councillor | leader_name = | area_metro_km2 = 6298 | population_as_of = 2011 | population_metro_footnotes = | population_metro = 2921488 | population_density_metro_km2 = auto | elevation_m = 1339 <!-- blank fields (section 2) --> | blank_name_sec2 = GDP | blank_info_sec2 = [[American dollar|US$]] 49.9 billion {{cite web|url=http://www.brookings.edu/research/interactives/global-metro-monitor-3|title=Global city GDP 2011|publisher=Brookings Institution|accessdate=18 November 2014}} |blank_name_sec1 = [[Human Development Index|HDI]] |blank_info_sec1 = {{increase}} 0.75 '''High''' (2012) {{cite web | url=http://www.gcro.ac.za/sites/default/files/maps/pdfs/gcro_map_of_the_month_hdi_1996_to_2012_april_2014.pdf | title=Gauteng's Human Development Index | publisher=Gauteng City-Region Observatory | date=2013 | accessdate=1 January 2015 | pages=1}} | blank1_name_sec2 = GDP per capita | blank1_info_sec2 = US$ 16,696 | blank2_name_sec2 = | blank2_info_sec2 = <!-- etc., up to blank7_name_sec2 / blank7_info_sec2 --> | postal_code = | area_code = [[011 (South Africa Calling Code)|012]] | censuscode = 799035 | website = http://www.tshwane.gov.za/Pages/default.aspx | footnotes = }} '''Pretoria''' is a city in the northern part of [[Gauteng Province]], [[South Africa]]. It is one of the country's three [[Capital (political)|capital cities]], serving as the executive (administrative) and de facto national capital ([[Cape Town]] is the [[legislature|legislative]] capital and [[Bloemfontein]] the [[judicial]] capital). Pretoria is the central part of the [[City of Tshwane Metropolitan Municipality]] which was formed by the amalgamation of several former local authorities including [[Centurion, Gauteng|Centurion]] and [[Soshanguve]]). There have been proposals to change the name of Pretoria itself to Tshwane, and the [[Pretoria#Change of name|proposed name change]] has caused some controversy. Pretoria is named after the [[Voortrekker]] leader [[Andries Pretorius]], {{cite book|last=Raper|first=Peter E.|title=Dictionary of Southern African Place Names|url=http://archive.org/stream/DictionaryOfSouthernAfricanPlaceNames/SaPlaceNames#page/n373/mode/2up|publisher=Internet Archive|accessdate=28 August 2013|page=373|year=1987}} and within South Africa is popularly known as the Jacaranda City due to the thousands of [[Jacaranda]] trees planted in its streets, parks and gardens. {{cite web |url=http://www.info.gov.za/aboutsa/provinces.htm#gauteng |title=South Africa's provinces: Gauteng |accessdate=14 June 2011}} ==History== [[File:Pretorius.jpg|thumb|upright|left|Statue of Andries Wilhelmus Jacobus Pretorius (27 November 1798 – 23 July 1853) in Pretoria]] [[File:Union-buildings-pretoria.JPG|thumb|The [[Union Buildings]], seat of South Africa's government]] [[File:Palace of Justice - Pretoria.jpg|thumb|The Palace of J

In [0]:
source_df.write.format("delta").option("delta.enableChangeDataFeed", "true").saveAsTable(source_table_fullname)

In [0]:
display(spark.sql(f"SELECT * FROM {source_table_fullname}"))

title id revisionId revisionTimestamp revisionUsername revisionUsernameId text Emil Kraepelin 10002 677433200 2015-08-23T06:55:55Z Yobot 7328338 {{Use dmy dates|date=July 2012}} {{Infobox scientist | name = Emil Kraepelin | image = Emil Kraepelin 1926.jpg | image_size = | alt = | caption = Emil Kraepelin in his later years | birth_date = {{Birth date|df=yes|1856|02|15}} | birth_place = [[Neustrelitz]] | death_date = {{Death date and age|df=yes|1926|10|07|1856|02|15}} | death_place = [[Munich]] | resting_place = | resting_place_coordinates = <!-- {{Coord|LAT|LONG|type:landmark|display=inline,title}} --> | residence = | citizenship = | nationality = [[German people|German]] | fields = [[Psychiatry]] | workplaces = [[University of Dorpat]], [[University of Heidelberg]], [[Ludwig Maximilian University of Munich|University of Munich]] | alma_mater = [[Leipzig University]], [[University of Würzburg]], [[Ludwig Maximilian University of Munich|University of Munich]] | thesis_title = The Place of Psychology in Psychiatry | thesis_url = | thesis_year = | doctoral_advisor = | academic_advisors = | doctoral_students = | notable_students = | known_for = [[Classification of mental disorders]], [[Kraepelinian dichotomy]] | author_abbrev_bot = | author_abbrev_zoo = | influences = [[Wilhelm Wundt]], [[Bernhard von Gudden]], [[Karl Ludwig Kahlbaum]] | influenced = [[Diagnostic and Statistical Manual of Mental Disorders]], [[International Statistical Classification of Diseases and Related Health Problems]] | awards = | signature = Emil_Kraepelin_signature.JPG | signature_alt = | website = | footnotes = | spouse = }} '''Emil Kraepelin''' (/ˈɛ:.mil ˈkrɛ:pɛ.lin/; 15 February 1856 – 7 October 1926) was a [[Germany|German]] [[psychiatrist]]. [[Hans Eysenck|H. J. Eysenck]]'s ''Encyclopedia of Psychology'' identifies him as the founder of modern scientific [[psychiatry]], as well as of [[psychopharmacology]] and [[psychiatric genetics]]. Kraepelin believed the chief origin of psychiatric [[disease]] to be [[biological process|biological]] and [[Genetics|genetic]] malfunction. His theories dominated psychiatry at the start of the 20th century and, despite the later [[psychodynamic]] influence of [[Sigmund Freud]] and his disciples, enjoyed a revival at century's end. == Family and early life == Kraepelin, whose father was a former opera singer, music teacher, and later successful story teller, Peter Barham (2004), ''Forgotten Lunatics of the Great War'' (New Haven: Yale) p. 163 n. 47. was born in 1856 in [[Neustrelitz]], in the [[Duchy of Mecklenburg-Strelitz]] in [[Germany]]. He was first introduced to biology by his brother Karl, 10 years older and, later, the director of the Zoological Museum of Hamburg. On Kraepelin's early life and family, see Burgmair et al., vol. I, as well as his ''Memoirs'' (Berlin: Springer, 1987). ==Career== [[File:Kraepelin.JPG|thumb|Grave in Heidelberg (2008)]] Kraepelin began his medical studies at 18 at the [[University of Leipzig]] and completed them at the [[University of Würzburg]]. At [[Leipzig]], he studied neuropathology under [[Paul Flechsig]] and experimental psychology with [[Wilhelm Wundt]]. Kraepelin would be a disciple of Wundt and had a lifelong interest in [[experimental psychology]] based on his theories. While there, Kraepelin wrote a prize-winning essay, "The Influence of Acute Illness in the Causation of Mental Disorders", and at Würzburg he received his MD in 1878. In 1879, he went to work with [[Bernhard von Gudden]] at the University of Munich, where he completed his habilitation thesis, "The Place of Psychology in Psychiatry". Returning to the University of Leipzig in 1882, he worked in [[Wilhelm Heinrich Erb]]'s neurology clinic and in Wundt's psychopharmacology laboratory. [http://www.encyclopedia.com/topic/Emil_Kraepelin.aspx#2 Kraepelin, Emil (1856–1926)] by Margaret Alic, Gale Encyclopedia of Psychology, 2001. Kraepelin's major work, ''Compendium der Psychiatrie: Zum Gebrauche für Studirende und


## Create vector search endpoint

In [0]:
vector_search_endpoint_name = "vector-search-demo-endpoint"

In [0]:
try:
    vsc.create_endpoint(
        name=vector_search_endpoint_name,
        endpoint_type="STANDARD" # or "STORAGE_OPTIMIZED"
    )
except Exception as e:
    if "ALREADY_EXISTS" in str(e):
        print("Endpoint already exists, continuing...")
    else:
        raise e

Endpoint already exists, continuing...


In [0]:
endpoint = vsc.get_endpoint(
  name=vector_search_endpoint_name)
endpoint

{'name': 'vector-search-demo-endpoint',
 'creator': 'fernando.rodriguez@databricks.com',
 'creation_timestamp': 1750714567884,
 'last_updated_timestamp': 1750714567884,
 'endpoint_type': 'STANDARD',
 'last_updated_user': 'fernando.rodriguez@databricks.com',
 'id': '52914a6d-d46a-4ff9-a704-3385398b490e',
 'endpoint_status': {'state': 'ONLINE'},
 'num_indexes': 0,
 'effective_budget_policy_id': '45e016d4-ace8-407d-870f-3f06123daee1'}

## Create vector index

In [0]:
# Vector index
vs_index = "en_wiki_index"
vs_index_fullname = f"{catalog_name}.{schema_name}.{vs_index}"

embedding_model_endpoint = "databricks-gte-large-en"

In [0]:
index = vsc.create_delta_sync_index(
  endpoint_name=vector_search_endpoint_name,
  source_table_name=source_table_fullname,
  index_name=vs_index_fullname,
  pipeline_type='TRIGGERED',
  primary_key="id",
  embedding_source_column="text",
  embedding_model_endpoint_name=embedding_model_endpoint
)
index.describe()

{'name': 'main.default.en_wiki_index',
 'endpoint_name': 'vector-search-demo-endpoint',
 'primary_key': 'id',
 'index_type': 'DELTA_SYNC',
 'delta_sync_index_spec': {'source_table': 'main.default.en_wiki',
  'embedding_source_columns': [{'name': 'text',
    'embedding_model_endpoint_name': 'databricks-gte-large-en'}],
  'pipeline_type': 'TRIGGERED',
  'pipeline_id': '014ad73e-9533-4fc2-bba2-b6bec941da4d'},
 'status': {'detailed_state': 'PROVISIONING_INDEX',
  'message': 'Delta sync Index creation is pending. Check latest status: https://e2-demo-field-eng.cloud.databricks.com/explore/data/main/default/en_wiki_index',
  'indexed_row_count': 0,
  'ready': False,
  'index_url': 'e2-demo-field-eng.cloud.databricks.com/api/2.0/vector-search/indexes/main.default.en_wiki_index'},
 'creator': 'prasad.kona@databricks.com',
 'endpoint_type': 'STANDARD'}

## Get a vector index  

Use `get_index()` to retrieve the vector index object using the vector index name. You can also use `describe()` on the index object to see a summary of the index's configuration information.

In [0]:


index = vsc.get_index(endpoint_name=vector_search_endpoint_name, index_name=vs_index_fullname)

index.describe()

{'name': 'main.default.en_wiki_index',
 'endpoint_name': 'vector-search-demo-endpoint',
 'primary_key': 'id',
 'index_type': 'DELTA_SYNC',
 'delta_sync_index_spec': {'source_table': 'main.default.en_wiki',
  'embedding_source_columns': [{'name': 'text',
    'embedding_model_endpoint_name': 'databricks-gte-large-en'}],
  'pipeline_type': 'TRIGGERED',
  'pipeline_id': '014ad73e-9533-4fc2-bba2-b6bec941da4d'},
 'status': {'detailed_state': 'PROVISIONING_INITIAL_SNAPSHOT',
  'message': 'Index is currently is in the process of syncing initial data. Check latest status: https://e2-demo-field-eng.cloud.databricks.com/explore/data/main/default/en_wiki_index',
  'indexed_row_count': 0,
  'provisioning_status': {'initial_pipeline_sync_progress': {'latest_version_currently_processing': 0,
    'num_synced_rows': 0,
    'total_rows_to_sync': 0,
    'sync_progress_completion': 1.0,
    'estimated_completion_time_seconds': 0.0,
    'pipeline_metrics': {'total_sync_time_per_row_ms': 0.0,
     'ingestio

In [0]:
# Wait for index to come online. Expect this command to take several minutes.
import time
while not index.describe().get('status').get('detailed_state').startswith('ONLINE'):
  print("Waiting for index to be ONLINE...")
  time.sleep(5)
print("Index is ONLINE")
index.describe()

Index is ONLINE


{'name': 'main.default.en_wiki_index',
 'endpoint_name': 'vector-search-demo-endpoint',
 'primary_key': 'id',
 'index_type': 'DELTA_SYNC',
 'delta_sync_index_spec': {'source_table': 'main.default.en_wiki',
  'embedding_source_columns': [{'name': 'text',
    'embedding_model_endpoint_name': 'databricks-gte-large-en'}],
  'pipeline_type': 'TRIGGERED',
  'pipeline_id': '014ad73e-9533-4fc2-bba2-b6bec941da4d'},
 'status': {'detailed_state': 'ONLINE_NO_PENDING_UPDATE',
  'message': 'Index creation succeeded. Check latest status: https://e2-demo-field-eng.cloud.databricks.com/explore/data/main/default/en_wiki_index',
  'indexed_row_count': 10,
  'triggered_update_status': {'last_processed_commit_version': 0,
   'last_processed_commit_timestamp': '2025-06-24T17:26:09Z'},
  'ready': True,
  'index_url': 'e2-demo-field-eng.cloud.databricks.com/api/2.0/vector-search/indexes/main.default.en_wiki_index'},
 'creator': 'prasad.kona@databricks.com',
 'endpoint_type': 'STANDARD'}

# Update Index and trigger vector search index sync


A function to Update the Delta table and triggers a vector index sync in triggered mode.

In [0]:
import time
from databricks.vector_search.client import VectorSearchClient

def update_table_and_trigger_sync(spark, update_query, endpoint_name, index_name):
    """
    Updates the Delta table and triggers a vector index sync in triggered mode.
    Ensures that no concurrent syncs are running by checking status with describe().
    Waits for the index to be ONLINE and for the new sync to complete.
    Handles errors gracefully.
    """
    try:
        # Step 1: Update the Delta table
        spark.sql(update_query)
        print("Delta table updated successfully.")
    except Exception as e:
        print(f"Error updating Delta table: {str(e)}")
        return

    try:
        # Step 2: Initialize the Vector Search client and get the index
        client = VectorSearchClient()
        index = client.get_index(endpoint_name=endpoint_name, index_name=index_name)

        # Step 3: Wait for index to be ONLINE
        print("Checking if index is ONLINE...")
        while True:
            status = index.describe()
            detailed_state = (
                status.get('status', {}).get('detailed_state') or
                status.get('detailed_state')  # fallback if structure is different
            )
            print(f"Current detailed_state: {detailed_state}")
            if detailed_state and detailed_state.startswith('ONLINE'):
                print("Index is ONLINE.")
                break
            print("Waiting for index to be ONLINE...")
            time.sleep(5)
        print("Index is ONLINE. Full status:")
        print(index.describe())

        # Step 4: Check sync status and wait if a sync is running
        sync_status = status.get("sync_status") or status.get("status")
        if sync_status in ("RUNNING", "IN_PROGRESS"):
            print("A sync is currently running. Waiting for it to complete...")
            index.wait_until_ready(wait_for_updates=True)
            print("Previous sync completed. Ready for a new sync.")
        else:
            print("No sync currently running. Proceeding.")

        # Step 5: Trigger a new sync
        index.sync()
        print("Sync triggered. Waiting for sync to complete...")

        # Step 6: Wait for the new sync to finish
        index.wait_until_ready(wait_for_updates=True)
        print(f"Sync completed successfully for index: {index_name}")

    except Exception as e:
        print(f"Error during sync process for index {index_name}: {str(e)}")
        # Optional: Add logging, retries, or alerting here

# Example usage
update_query = f"UPDATE {source_table_fullname} SET text = 'new text' WHERE id = 10002"
endpoint_name = vector_search_endpoint_name
index_name = vs_index_fullname

update_table_and_trigger_sync(spark, update_query, endpoint_name, index_name)


Delta table updated successfully.
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
Checking if index is ONLINE...
Current detailed_state: ONLINE_NO_PENDING_UPDATE
Index is ONLINE.
Index is ONLINE. Full status:
{'name': 'main.default.en_wiki_index', 'endpoint_name': 'vector-search-demo-endpoint', 'primary_key': 'id', 'index_type': 'DELTA_SYNC', 'delta_sync_index_spec': {'source_table': 'main.default.en_wiki', 'embedding_source_columns': [{'name': 'text', 'embedding_model_endpoint_name': 'databricks-gte-large-en'}], 'pipeline_type': 'TRIGGERED', 'pipeline_id': '014ad73e-9533-4fc2-bba2-b6bec941da4d'}, 'status': {'detailed_state': 'ONLINE_NO_PENDING_UPDATE', 'message': 'Index creation succeeded. Check latest status: https://e2-demo-field-eng.cloud.databricks.com/explore/data/main/default/en_wiki_index', 'indexed_row_count': 10, 'tr

## Similarity search

Query the Vector Index to find similar documents.

In [0]:
# Returns [col1, col2, ...]
# You can set this to any subset of the columns.
all_columns = spark.table(source_table_fullname).columns

results = index.similarity_search(
  query_text="Greek myths",
  columns=all_columns,
  num_results=2)

results

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


{'manifest': {'column_count': 8,
  'columns': [{'name': 'title'},
   {'name': 'id'},
   {'name': 'revisionId'},
   {'name': 'revisionTimestamp'},
   {'name': 'revisionUsername'},
   {'name': 'revisionUsernameId'},
   {'name': 'text'},
   {'name': 'score'}]},
 'result': {'row_count': 2,
  'data_array': [['Anthem of Europe',
    10026.0,
    682266076.0,
    '2015-09-22T16:32:16',
    'Iridescent',
    937705.0,
    '{{About|the anthem adopted by the European Union and Council of Europe|the German poem|Ode to Joy|the composition|Symphony No. 9 (Beethoven)|the anthem of Kosovo|Europe (anthem)}} {{EngvarB|date=May 2013}} {{Use dmy dates|date=May 2013}} {{Infobox anthem |title = Ode to Joy |alt_title = European Anthem<ref name="autogenerated1">{{cite web|url=http://www.coe.int/T/E/Com/About_Coe/emblems/emblemes.asp|title=Emblemes|archiveurl=http://web.archive.org/web/20080822114923/www.coe.int/T/E/Com/About_Coe/emblems/emblemes.asp|archivedate=22 August 2008|accessdate=2010-08-29|publisher=

In [0]:
#Print the data from the vs index to verify that the data was updated

# Perform a scan
results = index.scan(num_results=10)

# Extract the values for keys 'id' and 'text' from results['data'], truncating 'text' to 100 characters
values = [
    {
        field['key']: (
            field['value']['string_value'][:100] if field['key'] == 'text' else field['value']
        ) if field['key'] == 'text' else (
            field['value']['number_value'] if field['key'] == 'id' else field['value']
        )
        for field in item['fields'] if field['key'] in ('id', 'text')
    }
    for item in results['data']
]

values

[{'id': 10002.0, 'text': 'new text'},
 {'id': 10003.0,
  'text': '{{Refimprove|date=July 2014}} [[Image:Evoluon logos.png|thumb|right|250px|Current (left) logo of the'},
 {'id': 10005.0,
  'text': "{{Progressivism}}  '''Progressive education''' is a [[pedagogical]] movement that began in the late "},
 {'id': 10008.0,
  'text': '{{Other uses}} {{Refimprove|article|date=January 2010}} [[File:Arc welding electrodes and electrode '},
 {'id': 10013.0,
  'text': "'''Evidence-based medicine''' ('''EBM''') is an approach to medical practice intended to optimize [["},
 {'id': 10018.0,
  'text': '{{Infobox scientist | name                    = Edsger Wybe Dijkstra | image                   = Eds'},
 {'id': 10021.0,
  'text': '{{tone|date=March 2012}} [[Perennialist]]s believe that one should teach the things that one deems t'},
 {'id': 10024.0,
  'text': '{{distinguish2|other meanings of the word [[Ecstasy (disambiguation)|Ecstasy]]}} {{Use dmy dates|dat'},
 {'id': 10025.0,
  'text': '{{for|a ga

In [0]:
# Search with a filter. Note that the syntax depends on the endpoint type.

# Standard endpoint syntax
results = index.similarity_search(
  query_text="Greek myths",
  columns=all_columns,
  filters={"id NOT": ("13770", "88231")},
  num_results=2)

# Storage-optimized endpoint syntax
# results = index.similarity_search(
#   query_text="Greek myths",
#   columns=all_columns,
#   filters='id NOT IN ("13770", "88231")',
#   num_results=2)

results


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


{'manifest': {'column_count': 8,
  'columns': [{'name': 'title'},
   {'name': 'id'},
   {'name': 'revisionId'},
   {'name': 'revisionTimestamp'},
   {'name': 'revisionUsername'},
   {'name': 'revisionUsernameId'},
   {'name': 'text'},
   {'name': 'score'}]},
 'result': {'row_count': 2,
  'data_array': [['Anthem of Europe',
    10026.0,
    682266076.0,
    '2015-09-22T16:32:16',
    'Iridescent',
    937705.0,
    '{{About|the anthem adopted by the European Union and Council of Europe|the German poem|Ode to Joy|the composition|Symphony No. 9 (Beethoven)|the anthem of Kosovo|Europe (anthem)}} {{EngvarB|date=May 2013}} {{Use dmy dates|date=May 2013}} {{Infobox anthem |title = Ode to Joy |alt_title = European Anthem<ref name="autogenerated1">{{cite web|url=http://www.coe.int/T/E/Com/About_Coe/emblems/emblemes.asp|title=Emblemes|archiveurl=http://web.archive.org/web/20080822114923/www.coe.int/T/E/Com/About_Coe/emblems/emblemes.asp|archivedate=22 August 2008|accessdate=2010-08-29|publisher=

## Convert results to LangChain documents

The first column retrieved is loaded into `page_content`, and the rest into metadata.

In [0]:
# You must have langchain installed on your cluster. The following command installs or upgrades langchain.
%pip install --upgrade langchain

In [0]:
from langchain.schema import Document
from typing import List

def convert_vector_search_to_documents(results) -> List[Document]:
  column_names = []
  for column in results["manifest"]["columns"]:
      column_names.append(column)

  langchain_docs = []
  for item in results["result"]["data_array"]:
      metadata = {}
      score = item[-1]
      # print(score)
      i = 1
      for field in item[1:-1]:
          # print(field + "--")
          metadata[column_names[i]["name"]] = field
          i = i + 1
      doc = Document(page_content=item[0], metadata=metadata)  # , 9)
      langchain_docs.append(doc)
  return langchain_docs

langchain_docs = convert_vector_search_to_documents(results)

langchain_docs

[Document(metadata={'id': 10026.0, 'revisionId': 682266076.0, 'revisionTimestamp': '2015-09-22T16:32:16', 'revisionUsername': 'Iridescent', 'revisionUsernameId': 937705.0, 'text': '{{About|the anthem adopted by the European Union and Council of Europe|the German poem|Ode to Joy|the composition|Symphony No. 9 (Beethoven)|the anthem of Kosovo|Europe (anthem)}} {{EngvarB|date=May 2013}} {{Use dmy dates|date=May 2013}} {{Infobox anthem |title = Ode to Joy |alt_title = European Anthem<ref name="autogenerated1">{{cite web|url=http://www.coe.int/T/E/Com/About_Coe/emblems/emblemes.asp|title=Emblemes|archiveurl=http://web.archive.org/web/20080822114923/www.coe.int/T/E/Com/About_Coe/emblems/emblemes.asp|archivedate=22 August 2008|accessdate=2010-08-29|publisher=coe.int}}</ref><ref name="autogenerated2">{{cite web|url=http://europa.eu/abc/symbols/anthem/index_en.htm|title=EUROPA – The EU at a glance – The European Anthem|accessdate=2010-08-29|publisher=europa.eu| archiveurl= http://web.archive.or

## Delete vector index

In [0]:
vsc.delete_index(index_name=vs_index_fullname)